In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [4]:
base_dir = r'E:\\Cutting Tool Paper\\Dataset\\cutting tool data\\new_data\\CWT'
classes = ['Bearing Fault Data', 'Gear Fault Data', 'Normal Data', 'Tool Fault Data']  # Replace with your actual class names


In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'train_test_split' is not defined

In [6]:
# Parameters
IMAGE_SIZE = (224, 224)  # Increase image size for better feature extraction
BATCH_SIZE = 16

# ImageDataGenerator for training and validation sets with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 128 images belonging to 4 classes.
Found 32 images belonging to 4 classes.


In [7]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [9]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.regularizers import l2

pretrained_model = InceptionV3(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
pretrained_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 249

for layer in pretrained_model.layers[:fine_tune_at]:
    layer.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)




In [10]:
EPOCHS = 50

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_scheduler]
)


Epoch 1/50


C:\Users\Muhammad Umar\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.3225 - loss: 9.4535 - val_accuracy: 0.7188 - val_loss: 8.8287 - learning_rate: 1.0000e-04
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 544ms/step - accuracy: 0.8092 - loss: 8.6011 - val_accuracy: 0.7188 - val_loss: 8.5887 - learning_rate: 1.0000e-04
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 569ms/step - accuracy: 0.9173 - loss: 8.2376 - val_accuracy: 0.8125 - val_loss: 8.2841 - learning_rate: 1.0000e-04
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 512ms/step - accuracy: 0.9183 - loss: 8.0262 - val_accuracy: 0.7500 - val_loss: 8.3176 - learning_rate: 1.0000e-04
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 524ms/step - accuracy: 0.9460 - loss: 7.8366 - val_accuracy: 0.7500 - val_loss: 8.2095 - learning_rate: 1.0000e-04


ValueError: The output of the `schedule` function should be a float. Got: 9.048374340636656e-05

In [ ]:
# Plotting training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [ ]:
# Compute Confusion Matrix for Training Data
train_generator.reset()
train_labels = []
train_predictions = []

for i in range(len(train_generator)):
    images, labels = next(train_generator)
    preds = model.predict(images)
    train_labels.extend(np.argmax(labels, axis=1))
    train_predictions.extend(np.argmax(preds, axis=1))

train_cm = confusion_matrix(train_labels, train_predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Training Data')
plt.show()


In [ ]:
# Compute Confusion Matrix for Validation Data
validation_generator.reset()
val_labels = []
val_predictions = []

for i in range(len(validation_generator)):
    images, labels = next(validation_generator)
    preds = model.predict(images)
    val_labels.extend(np.argmax(labels, axis=1))
    val_predictions.extend(np.argmax(preds, axis=1))

val_cm = confusion_matrix(val_labels, val_predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Validation Data')
plt.show()
